# FDCHP Data Processing

This Notebook demonstrates how to process FDCHP data using the tools in the `ooi-data-explorations.python.examples.fdchp` library.

This requires setting up a python environment that contains the necessary dependencies to use in this notebook.

Alongside the base repository containing this Notebook (the location of the `ooi-data-explorations` repository, clone the `mi-instrument` repo 

```
git clone https://github.com/joffreyp/mi-instrument.git
```

Create a new python virtual environment:

```
cd ~
mkdir .venv
python -m venv ~/.venv
source .venv/bin/activate
```

Install mi-instrument requirements:

```
pip install -r mi-instrument/requirements_py3.txt
```

Install `ooi-data-explorations` and `mi-instruments`: 

```
pip install -e ooi-data-explorations/python/
pip install -e mi-instrument/
```

Prepare environment for use by Jupyter Notebook kernel:

```
pip install ipykernel
python -m ipykernel install --user --name=fdchp
vi ~/.local/share/jupyter/kernels/fdchp/kernel.json
```

And replace the first line of the `argv` array in `kernel.json` with the python from the new .venv: `"~/.venv/bin/python",`

Now, select the `Kernel` menu, click `Change kernel...` and select the new `fdchp` environment.


## Import necessary libraries

In [1]:
import pandas as pd
import mi
from datetime import datetime
import numpy as np

from examples.fdchp import fdchp_utils
from examples.fdchp.fdchp_utils import particles_to_pandas, read_file, read_file_to_pandas
from examples.fdchp.process_fdchp import process_fdchp


/home/jovyan/FDCHPTest/mi-instrument/mi/core/log.py:108: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  parsed = yaml.load(logconfig)


## Select and load dataset from raw data server

In [2]:
import os
from glob import glob

data_directory = '/home/jovyan/ooi/uncabled/GI01SUMO/R00008/instruments/dcl12/FDCHP_sn144904/D202110'
files = glob(os.path.join(data_directory, '*.dat'))

data = fdchp_utils.read_file(files[0])

## Set up some variables for FDCHP processing

In [3]:
instrument_dir = data_directory.split('uncabled')[-1]
if instrument_dir.startswith('PA'):
    lat=40.1334        #Pioneer NES
    instrument_rel_position=[-0.75, 0, -5]
elif instrument_dir.startswith('IS'):
    lat=59.9337        #Irminger Sea
    instrument_rel_position=[-0.75, 0, -6] 
else:
    lat=44.6393        #Endurance
    instrument_rel_position=[-0.5, -0.5, -5]

U=[]
uw=[]
sigH=[]
data = []

## Process all the files in the data directory

In [ ]:
output_filepath = "fluxes"
output_filename = "fluxes{}"

if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
print("Processing {} files.".format(len(files)))
incr=0
print("Start time: {}".format(datetime.now()))
# for incr in np.arange(24):
for filename in files:
    #****************************************
    # Read in raw data
    #**********************************
    ******
    start = datetime.now()
    # if incr <= 9:
    #     filename = os.path.join(file_path, 'fdchp_20200818_0'+ str(incr)+ '0200.dat')
    # else:
    #     filename = os.path.join(file_path, 'fdchp_20200818_'+ str(incr)+ '0200.dat')

    print("Processing file {}: {}".format(incr + 1, filename))
    try:
        data = read_file(filename)
    except Exception as e:
        print("Error reading in file: {}".format(e))
        continue
              
    raw_data = particles_to_pandas(data)

    # raw_data = read_file_to_pandas(filename)

    data_readin = datetime.now()
    
    #*****************************************
    # Compute flux data
    #*****************************************   
    try:
        fluxes, Uearth, waveheight = process_fdchp(raw_data, lat, instrument_rel_position, flux_filepath=os.path.join(output_filepath, output_filename.format(incr)))
    except Exception as e:
        # Error processing data; probably too few datapoints
        print("Error processing fdchp dataset: {}".format(e))
        continue
    processed = datetime.now()
    print("File processing finished. Read-in time: {}, Process time: {}".format(data_readin-start, processed-data_readin))
    uw = uw + [-fluxes[0]]        # Fluxes: uw vw wT
    U = U + [Uearth]              # Wind speed relative to earth  
    sigH = sigH + [waveheight]    # Significant wave height
    incr += 1

uw = np.array(uw)
U = np.array(U)
sigH = np.array(sigH)

print("End time: {}".format(datetime.now()))


Processing 413 files.
Start time: 2024-09-17 15:04:01.456919
Processing file 1: /home/jovyan/ooi/uncabled/GI01SUMO/R00008/instruments/dcl12/FDCHP_sn144904/D202110/fdchp_20211003_180200.dat
File processing finished. Read-in time: 0:00:08.980017, Process time: 0:00:00.407693
Processing file 2: /home/jovyan/ooi/uncabled/GI01SUMO/R00008/instruments/dcl12/FDCHP_sn144904/D202110/fdchp_20211003_220200.dat
File processing finished. Read-in time: 0:00:08.733232, Process time: 0:00:00.064535
Processing file 3: /home/jovyan/ooi/uncabled/GI01SUMO/R00008/instruments/dcl12/FDCHP_sn144904/D202110/fdchp_20211007_130200.dat
File processing finished. Read-in time: 0:00:08.674933, Process time: 0:00:00.069960
Processing file 4: /home/jovyan/ooi/uncabled/GI01SUMO/R00008/instruments/dcl12/FDCHP_sn144904/D202110/fdchp_20211005_160200.dat
File processing finished. Read-in time: 0:00:08.828167, Process time: 0:00:00.064220
Processing file 5: /home/jovyan/ooi/uncabled/GI01SUMO/R00008/instruments/dcl12/FDCHP_sn

## Plot the data

In [ ]:
from plot_fdchp import plot_x_velocity_vs_wind_speed, plot_wave_height_vs_wind_speed

plot_x_velocity_vs_wind_speed(U, uw)

plot_wave_height_vs_wind_speed(U, sigH)